# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [109]:
import pandas as pd

import statsmodels.formula.api as smf
import statsmodels.api as sm

import numpy as np

from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [33]:
df = pd.read_csv('previsao_de_renda.csv') #Carregando dados

In [3]:
df.head(3) #Visualizando parte dos dados

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89


#### Tratando dados

In [34]:
df = df.drop(['Unnamed: 0', 'data_ref', 'id_cliente'], axis = 1) #Removendo variáveis irrelevantes

In [5]:
df.head(1)

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.60274,1.0,8060.34


In [39]:
df.isna().sum() #Verificando dados ausentes

sexo                     0
posse_de_veiculo         0
posse_de_imovel          0
qtd_filhos               0
tipo_renda               0
educacao                 0
estado_civil             0
tipo_residencia          0
idade                    0
tempo_emprego            0
qt_pessoas_residencia    0
renda                    0
dtype: int64

- Encontrei dados ausentes na coluna 'tempo_emprego' e, para decir como tratar isso, resolvi visualizar o tipo de renda das pessoas que apresentaram essa característica.

In [36]:
nulos = df[df.tempo_emprego.isnull()] # Capturando todas e somente as linhas que apresentaram dados faltantes

In [8]:
nulos['tipo_renda'].unique() #Exibindo quais tipos de renda possuem as pessoas com valores nulos em 'tempo_emprego'

array(['Pensionista'], dtype=object)

- Após análise, foi possível chegar à conclusão de que todas as pessoas sem tempo de emprego cadastrado são pensionistas e, por isso, os valores nulos serão preenchidos por 0 e as linhas serão mantidas no dataframe.

In [38]:
df = df.fillna(0) #Preenchendo dados faltantes

#### Tarefas

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).

In [10]:
df_train = df.sample(frac=0.75, random_state=1) #Criando df de treinamento com 75% dos dados

In [11]:
df_test = df.drop(df_train.index) #Criando df de teste com 25% dos dados restantes

2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?

In [12]:
#  Listando variáveis (resposta e explicativas).
list_data = 'renda ~ C(sexo) + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia'

In [13]:
model = smf.ols(list_data, data = df_test) # Criando modelo

In [14]:
reg = model.fit_regularized(method = 'elastic_net' # Treinando e regularizando modelo
                        , refit = True
                        , L1_wt = 0.01
                        , alpha = 0)

In [15]:
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.253
Model:                            OLS   Adj. R-squared:                  0.249
Method:                 Least Squares   F-statistic:                     52.62
Date:                Mon, 04 Sep 2023   Prob (F-statistic):          4.65e-215
Time:                        09:49:58   Log-Likelihood:                -38891.
No. Observations:                3750   AIC:                         7.783e+04
Df Residuals:                    3726   BIC:                         7.799e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                         3316.5655   7192.099      0.461      0.645   -1.08e+04    1.74e+04
C(sexo)[T.M]                      6385.4406    298.547     21.388      0.000    5800.109    6970.772
posse_de_veiculo[T.True]          -423.1622    282.950     -1.496      0.135    -977.915     131.590
posse_de_imovel[T.True]            225.5227    278.700      0.809      0.418    -320.897     771.943
tipo_renda[T.Empresário]           926.2318    323.760      2.861      0.004     291.467    1560.996
tipo_renda[T.Pensionista]         2690.3196    519.634      5.177      0.000    1671.524    3709.115
tipo_renda[T.Servidor público]    -128.5786    472.495     -0.272      0.786   -1054.953     797.796
educacao[T.Pós graduação]         1680.8059   3406.485      0.493      0.622   -4997.952    8359.564
educacao[T.Secundário]             498.2077   1257.359      0.396      0.692   -1966.972    2963.387
educacao[T.Superior completo]      723.6434   1265.240      0.572      0.567   -1756.987    3204.274
educacao[T.Superior incompleto]   -181.5190   1408.601     -0.129      0.897   -2943.223    2580.185
estado_civil[T.Separado]         -2824.7662   3533.037     -0.800      0.424   -9751.641    4102.109
estado_civil[T.Solteiro]         -3647.4307   3475.884     -1.049      0.294   -1.05e+04    3167.390
estado_civil[T.União]             -448.9957    514.868     -0.872      0.383   -1458.447     560.455
estado_civil[T.Viúvo]            -3542.9743   3548.949     -0.998      0.318   -1.05e+04    3415.098
tipo_residencia[T.Casa]            -94.7471   1149.074     -0.082      0.934   -2347.622    2158.128
tipo_residencia[T.Com os pais]     -15.1621   1277.870     -0.012      0.991   -2520.556    2490.232
tipo_residencia[T.Comunitário]     355.3477   2208.104      0.161      0.872   -3973.864    4684.559
tipo_residencia[T.Estúdio]        1856.0739   2008.462      0.924      0.355   -2081.719    5793.867
tipo_residencia[T.Governamental]   477.2171   1365.617      0.349      0.727   -2200.214    3154.648
qtd_filhos                        2953.1481   3505.619      0.842      0.400   -3919.972    9826.268
idade                               41.8500     16.178      2.587      0.010      10.130      73.569
tempo_emprego                      576.5410     22.251     25.911      0.000     532.916     620.166
qt_pessoas_residencia            -3010.0789   3495.813     -0.861      0.389   -9863.973    3843.815
==============================================================================
Omnibus:                     5313.243   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2370379.061
Skew:                           8.130   Prob(JB):                         0.00
Kurtosis:                     125.090   

In [16]:
reg = model.fit_regularized(method = 'elastic_net' # Treinando e regularizando modelo
                        , refit = True
                        , L1_wt = 0.01
                        , alpha = 0.001)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.253
Model:                            OLS   Adj. R-squared:                  0.249
Method:                 Least Squares   F-statistic:                     52.62
Date:                Mon, 04 Sep 2023   Prob (F-statistic):          4.65e-215
Time:                        09:50:02   Log-Likelihood:                -38891.
No. Observations:                3750   AIC:                         7.783e+04
Df Residuals:                    3726   BIC:                         7.799e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                         3316.5655   7192.099      0.461      0.645   -1.08e+04    1.74e+04
C(sexo)[T.M]                      6385.4406    298.547     21.388      0.000    5800.109    6970.772
posse_de_veiculo[T.True]          -423.1622    282.950     -1.496      0.135    -977.915     131.590
posse_de_imovel[T.True]            225.5227    278.700      0.809      0.418    -320.897     771.943
tipo_renda[T.Empresário]           926.2318    323.760      2.861      0.004     291.467    1560.996
tipo_renda[T.Pensionista]         2690.3196    519.634      5.177      0.000    1671.524    3709.115
tipo_renda[T.Servidor público]    -128.5786    472.495     -0.272      0.786   -1054.953     797.796
educacao[T.Pós graduação]         1680.8059   3406.485      0.493      0.622   -4997.952    8359.564
educacao[T.Secundário]             498.2077   1257.359      0.396      0.692   -1966.972    2963.387
educacao[T.Superior completo]      723.6434   1265.240      0.572      0.567   -1756.987    3204.274
educacao[T.Superior incompleto]   -181.5190   1408.601     -0.129      0.897   -2943.223    2580.185
estado_civil[T.Separado]         -2824.7662   3533.037     -0.800      0.424   -9751.641    4102.109
estado_civil[T.Solteiro]         -3647.4307   3475.884     -1.049      0.294   -1.05e+04    3167.390
estado_civil[T.União]             -448.9957    514.868     -0.872      0.383   -1458.447     560.455
estado_civil[T.Viúvo]            -3542.9743   3548.949     -0.998      0.318   -1.05e+04    3415.098
tipo_residencia[T.Casa]            -94.7471   1149.074     -0.082      0.934   -2347.622    2158.128
tipo_residencia[T.Com os pais]     -15.1621   1277.870     -0.012      0.991   -2520.556    2490.232
tipo_residencia[T.Comunitário]     355.3477   2208.104      0.161      0.872   -3973.864    4684.559
tipo_residencia[T.Estúdio]        1856.0739   2008.462      0.924      0.355   -2081.719    5793.867
tipo_residencia[T.Governamental]   477.2171   1365.617      0.349      0.727   -2200.214    3154.648
qtd_filhos                        2953.1481   3505.619      0.842      0.400   -3919.972    9826.268
idade                               41.8500     16.178      2.587      0.010      10.130      73.569
tempo_emprego                      576.5410     22.251     25.911      0.000     532.916     620.166
qt_pessoas_residencia            -3010.0789   3495.813     -0.861      0.389   -9863.973    3843.815
==============================================================================
Omnibus:                     5313.243   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2370379.061
Skew:                           8.130   Prob(JB):                         0.00
Kurtosis:                     125.090   

In [17]:
reg = model.fit_regularized(method = 'elastic_net' # Treinando e regularizando modelo
                        , refit = True
                        , L1_wt = 0.01
                        , alpha = 0.005)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.253
Model:                            OLS   Adj. R-squared:                  0.249
Method:                 Least Squares   F-statistic:                     52.62
Date:                Mon, 04 Sep 2023   Prob (F-statistic):          4.65e-215
Time:                        09:50:06   Log-Likelihood:                -38891.
No. Observations:                3750   AIC:                         7.783e+04
Df Residuals:                    3726   BIC:                         7.799e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                         3316.5655   7192.099      0.461      0.645   -1.08e+04    1.74e+04
C(sexo)[T.M]                      6385.4406    298.547     21.388      0.000    5800.109    6970.772
posse_de_veiculo[T.True]          -423.1622    282.950     -1.496      0.135    -977.915     131.590
posse_de_imovel[T.True]            225.5227    278.700      0.809      0.418    -320.897     771.943
tipo_renda[T.Empresário]           926.2318    323.760      2.861      0.004     291.467    1560.996
tipo_renda[T.Pensionista]         2690.3196    519.634      5.177      0.000    1671.524    3709.115
tipo_renda[T.Servidor público]    -128.5786    472.495     -0.272      0.786   -1054.953     797.796
educacao[T.Pós graduação]         1680.8059   3406.485      0.493      0.622   -4997.952    8359.564
educacao[T.Secundário]             498.2077   1257.359      0.396      0.692   -1966.972    2963.387
educacao[T.Superior completo]      723.6434   1265.240      0.572      0.567   -1756.987    3204.274
educacao[T.Superior incompleto]   -181.5190   1408.601     -0.129      0.897   -2943.223    2580.185
estado_civil[T.Separado]         -2824.7662   3533.037     -0.800      0.424   -9751.641    4102.109
estado_civil[T.Solteiro]         -3647.4307   3475.884     -1.049      0.294   -1.05e+04    3167.390
estado_civil[T.União]             -448.9957    514.868     -0.872      0.383   -1458.447     560.455
estado_civil[T.Viúvo]            -3542.9743   3548.949     -0.998      0.318   -1.05e+04    3415.098
tipo_residencia[T.Casa]            -94.7471   1149.074     -0.082      0.934   -2347.622    2158.128
tipo_residencia[T.Com os pais]     -15.1621   1277.870     -0.012      0.991   -2520.556    2490.232
tipo_residencia[T.Comunitário]     355.3477   2208.104      0.161      0.872   -3973.864    4684.559
tipo_residencia[T.Estúdio]        1856.0739   2008.462      0.924      0.355   -2081.719    5793.867
tipo_residencia[T.Governamental]   477.2171   1365.617      0.349      0.727   -2200.214    3154.648
qtd_filhos                        2953.1481   3505.619      0.842      0.400   -3919.972    9826.268
idade                               41.8500     16.178      2.587      0.010      10.130      73.569
tempo_emprego                      576.5410     22.251     25.911      0.000     532.916     620.166
qt_pessoas_residencia            -3010.0789   3495.813     -0.861      0.389   -9863.973    3843.815
==============================================================================
Omnibus:                     5313.243   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2370379.061
Skew:                           8.130   Prob(JB):                         0.00
Kurtosis:                     125.090   

In [18]:
reg = model.fit_regularized(method = 'elastic_net' # Treinando e regularizando modelo
                        , refit = True
                        , L1_wt = 0.01
                        , alpha = 0.01)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.253
Model:                            OLS   Adj. R-squared:                  0.249
Method:                 Least Squares   F-statistic:                     52.62
Date:                Mon, 04 Sep 2023   Prob (F-statistic):          4.65e-215
Time:                        09:50:09   Log-Likelihood:                -38891.
No. Observations:                3750   AIC:                         7.783e+04
Df Residuals:                    3726   BIC:                         7.799e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                         3316.5655   7192.099      0.461      0.645   -1.08e+04    1.74e+04
C(sexo)[T.M]                      6385.4406    298.547     21.388      0.000    5800.109    6970.772
posse_de_veiculo[T.True]          -423.1622    282.950     -1.496      0.135    -977.915     131.590
posse_de_imovel[T.True]            225.5227    278.700      0.809      0.418    -320.897     771.943
tipo_renda[T.Empresário]           926.2318    323.760      2.861      0.004     291.467    1560.996
tipo_renda[T.Pensionista]         2690.3196    519.634      5.177      0.000    1671.524    3709.115
tipo_renda[T.Servidor público]    -128.5786    472.495     -0.272      0.786   -1054.953     797.796
educacao[T.Pós graduação]         1680.8059   3406.485      0.493      0.622   -4997.952    8359.564
educacao[T.Secundário]             498.2077   1257.359      0.396      0.692   -1966.972    2963.387
educacao[T.Superior completo]      723.6434   1265.240      0.572      0.567   -1756.987    3204.274
educacao[T.Superior incompleto]   -181.5190   1408.601     -0.129      0.897   -2943.223    2580.185
estado_civil[T.Separado]         -2824.7662   3533.037     -0.800      0.424   -9751.641    4102.109
estado_civil[T.Solteiro]         -3647.4307   3475.884     -1.049      0.294   -1.05e+04    3167.390
estado_civil[T.União]             -448.9957    514.868     -0.872      0.383   -1458.447     560.455
estado_civil[T.Viúvo]            -3542.9743   3548.949     -0.998      0.318   -1.05e+04    3415.098
tipo_residencia[T.Casa]            -94.7471   1149.074     -0.082      0.934   -2347.622    2158.128
tipo_residencia[T.Com os pais]     -15.1621   1277.870     -0.012      0.991   -2520.556    2490.232
tipo_residencia[T.Comunitário]     355.3477   2208.104      0.161      0.872   -3973.864    4684.559
tipo_residencia[T.Estúdio]        1856.0739   2008.462      0.924      0.355   -2081.719    5793.867
tipo_residencia[T.Governamental]   477.2171   1365.617      0.349      0.727   -2200.214    3154.648
qtd_filhos                        2953.1481   3505.619      0.842      0.400   -3919.972    9826.268
idade                               41.8500     16.178      2.587      0.010      10.130      73.569
tempo_emprego                      576.5410     22.251     25.911      0.000     532.916     620.166
qt_pessoas_residencia            -3010.0789   3495.813     -0.861      0.389   -9863.973    3843.815
==============================================================================
Omnibus:                     5313.243   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2370379.061
Skew:                           8.130   Prob(JB):                         0.00
Kurtosis:                     125.090   

In [19]:
reg = model.fit_regularized(method = 'elastic_net' # Treinando e regularizando modelo
                        , refit = True
                        , L1_wt = 0.01
                        , alpha = 0.05)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.253
Model:                            OLS   Adj. R-squared:                  0.249
Method:                 Least Squares   F-statistic:                     52.62
Date:                Mon, 04 Sep 2023   Prob (F-statistic):          4.65e-215
Time:                        09:50:12   Log-Likelihood:                -38891.
No. Observations:                3750   AIC:                         7.783e+04
Df Residuals:                    3726   BIC:                         7.799e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                         3316.5655   7192.099      0.461      0.645   -1.08e+04    1.74e+04
C(sexo)[T.M]                      6385.4406    298.547     21.388      0.000    5800.109    6970.772
posse_de_veiculo[T.True]          -423.1622    282.950     -1.496      0.135    -977.915     131.590
posse_de_imovel[T.True]            225.5227    278.700      0.809      0.418    -320.897     771.943
tipo_renda[T.Empresário]           926.2318    323.760      2.861      0.004     291.467    1560.996
tipo_renda[T.Pensionista]         2690.3196    519.634      5.177      0.000    1671.524    3709.115
tipo_renda[T.Servidor público]    -128.5786    472.495     -0.272      0.786   -1054.953     797.796
educacao[T.Pós graduação]         1680.8059   3406.485      0.493      0.622   -4997.952    8359.564
educacao[T.Secundário]             498.2077   1257.359      0.396      0.692   -1966.972    2963.387
educacao[T.Superior completo]      723.6434   1265.240      0.572      0.567   -1756.987    3204.274
educacao[T.Superior incompleto]   -181.5190   1408.601     -0.129      0.897   -2943.223    2580.185
estado_civil[T.Separado]         -2824.7662   3533.037     -0.800      0.424   -9751.641    4102.109
estado_civil[T.Solteiro]         -3647.4307   3475.884     -1.049      0.294   -1.05e+04    3167.390
estado_civil[T.União]             -448.9957    514.868     -0.872      0.383   -1458.447     560.455
estado_civil[T.Viúvo]            -3542.9743   3548.949     -0.998      0.318   -1.05e+04    3415.098
tipo_residencia[T.Casa]            -94.7471   1149.074     -0.082      0.934   -2347.622    2158.128
tipo_residencia[T.Com os pais]     -15.1621   1277.870     -0.012      0.991   -2520.556    2490.232
tipo_residencia[T.Comunitário]     355.3477   2208.104      0.161      0.872   -3973.864    4684.559
tipo_residencia[T.Estúdio]        1856.0739   2008.462      0.924      0.355   -2081.719    5793.867
tipo_residencia[T.Governamental]   477.2171   1365.617      0.349      0.727   -2200.214    3154.648
qtd_filhos                        2953.1481   3505.619      0.842      0.400   -3919.972    9826.268
idade                               41.8500     16.178      2.587      0.010      10.130      73.569
tempo_emprego                      576.5410     22.251     25.911      0.000     532.916     620.166
qt_pessoas_residencia            -3010.0789   3495.813     -0.861      0.389   -9863.973    3843.815
==============================================================================
Omnibus:                     5313.243   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2370379.061
Skew:                           8.130   Prob(JB):                         0.00
Kurtosis:                     125.090   

In [20]:
reg = model.fit_regularized(method = 'elastic_net' # Treinando e regularizando modelo
                        , refit = True
                        , L1_wt = 0.01
                        , alpha = 0.1)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.253
Model:                            OLS   Adj. R-squared:                  0.249
Method:                 Least Squares   F-statistic:                     52.62
Date:                Mon, 04 Sep 2023   Prob (F-statistic):          4.65e-215
Time:                        09:50:14   Log-Likelihood:                -38891.
No. Observations:                3750   AIC:                         7.783e+04
Df Residuals:                    3726   BIC:                         7.799e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                         3316.5655   7192.099      0.461      0.645   -1.08e+04    1.74e+04
C(sexo)[T.M]                      6385.4406    298.547     21.388      0.000    5800.109    6970.772
posse_de_veiculo[T.True]          -423.1622    282.950     -1.496      0.135    -977.915     131.590
posse_de_imovel[T.True]            225.5227    278.700      0.809      0.418    -320.897     771.943
tipo_renda[T.Empresário]           926.2318    323.760      2.861      0.004     291.467    1560.996
tipo_renda[T.Pensionista]         2690.3196    519.634      5.177      0.000    1671.524    3709.115
tipo_renda[T.Servidor público]    -128.5786    472.495     -0.272      0.786   -1054.953     797.796
educacao[T.Pós graduação]         1680.8059   3406.485      0.493      0.622   -4997.952    8359.564
educacao[T.Secundário]             498.2077   1257.359      0.396      0.692   -1966.972    2963.387
educacao[T.Superior completo]      723.6434   1265.240      0.572      0.567   -1756.987    3204.274
educacao[T.Superior incompleto]   -181.5190   1408.601     -0.129      0.897   -2943.223    2580.185
estado_civil[T.Separado]         -2824.7662   3533.037     -0.800      0.424   -9751.641    4102.109
estado_civil[T.Solteiro]         -3647.4307   3475.884     -1.049      0.294   -1.05e+04    3167.390
estado_civil[T.União]             -448.9957    514.868     -0.872      0.383   -1458.447     560.455
estado_civil[T.Viúvo]            -3542.9743   3548.949     -0.998      0.318   -1.05e+04    3415.098
tipo_residencia[T.Casa]            -94.7471   1149.074     -0.082      0.934   -2347.622    2158.128
tipo_residencia[T.Com os pais]     -15.1621   1277.870     -0.012      0.991   -2520.556    2490.232
tipo_residencia[T.Comunitário]     355.3477   2208.104      0.161      0.872   -3973.864    4684.559
tipo_residencia[T.Estúdio]        1856.0739   2008.462      0.924      0.355   -2081.719    5793.867
tipo_residencia[T.Governamental]   477.2171   1365.617      0.349      0.727   -2200.214    3154.648
qtd_filhos                        2953.1481   3505.619      0.842      0.400   -3919.972    9826.268
idade                               41.8500     16.178      2.587      0.010      10.130      73.569
tempo_emprego                      576.5410     22.251     25.911      0.000     532.916     620.166
qt_pessoas_residencia            -3010.0789   3495.813     -0.861      0.389   -9863.973    3843.815
==============================================================================
Omnibus:                     5313.243   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2370379.061
Skew:                           8.130   Prob(JB):                         0.00
Kurtosis:                     125.090   

- Os valores de $R^2$ não apresentaram alterações.

3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?

In [21]:
reg = model.fit_regularized(method = 'elastic_net'
                           , refit = True
                           , L1_wt = 1
                           , alpha = 0)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.253
Model:                            OLS   Adj. R-squared:                  0.249
Method:                 Least Squares   F-statistic:                     52.62
Date:                Mon, 04 Sep 2023   Prob (F-statistic):          4.65e-215
Time:                        09:50:19   Log-Likelihood:                -38891.
No. Observations:                3750   AIC:                         7.783e+04
Df Residuals:                    3726   BIC:                         7.799e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                         3316.5655   7192.099      0.461      0.645   -1.08e+04    1.74e+04
C(sexo)[T.M]                      6385.4406    298.547     21.388      0.000    5800.109    6970.772
posse_de_veiculo[T.True]          -423.1622    282.950     -1.496      0.135    -977.915     131.590
posse_de_imovel[T.True]            225.5227    278.700      0.809      0.418    -320.897     771.943
tipo_renda[T.Empresário]           926.2318    323.760      2.861      0.004     291.467    1560.996
tipo_renda[T.Pensionista]         2690.3196    519.634      5.177      0.000    1671.524    3709.115
tipo_renda[T.Servidor público]    -128.5786    472.495     -0.272      0.786   -1054.953     797.796
educacao[T.Pós graduação]         1680.8059   3406.485      0.493      0.622   -4997.952    8359.564
educacao[T.Secundário]             498.2077   1257.359      0.396      0.692   -1966.972    2963.387
educacao[T.Superior completo]      723.6434   1265.240      0.572      0.567   -1756.987    3204.274
educacao[T.Superior incompleto]   -181.5190   1408.601     -0.129      0.897   -2943.223    2580.185
estado_civil[T.Separado]         -2824.7662   3533.037     -0.800      0.424   -9751.641    4102.109
estado_civil[T.Solteiro]         -3647.4307   3475.884     -1.049      0.294   -1.05e+04    3167.390
estado_civil[T.União]             -448.9957    514.868     -0.872      0.383   -1458.447     560.455
estado_civil[T.Viúvo]            -3542.9743   3548.949     -0.998      0.318   -1.05e+04    3415.098
tipo_residencia[T.Casa]            -94.7471   1149.074     -0.082      0.934   -2347.622    2158.128
tipo_residencia[T.Com os pais]     -15.1621   1277.870     -0.012      0.991   -2520.556    2490.232
tipo_residencia[T.Comunitário]     355.3477   2208.104      0.161      0.872   -3973.864    4684.559
tipo_residencia[T.Estúdio]        1856.0739   2008.462      0.924      0.355   -2081.719    5793.867
tipo_residencia[T.Governamental]   477.2171   1365.617      0.349      0.727   -2200.214    3154.648
qtd_filhos                        2953.1481   3505.619      0.842      0.400   -3919.972    9826.268
idade                               41.8500     16.178      2.587      0.010      10.130      73.569
tempo_emprego                      576.5410     22.251     25.911      0.000     532.916     620.166
qt_pessoas_residencia            -3010.0789   3495.813     -0.861      0.389   -9863.973    3843.815
==============================================================================
Omnibus:                     5313.243   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2370379.061
Skew:                           8.130   Prob(JB):                         0.00
Kurtosis:                     125.090   

In [22]:
reg = model.fit_regularized(method = 'elastic_net'
                           , refit = True
                           , L1_wt = 1
                           , alpha = 0.001)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.253
Model:                            OLS   Adj. R-squared:                  0.249
Method:                 Least Squares   F-statistic:                     52.62
Date:                Mon, 04 Sep 2023   Prob (F-statistic):          4.65e-215
Time:                        09:50:22   Log-Likelihood:                -38891.
No. Observations:                3750   AIC:                         7.783e+04
Df Residuals:                    3726   BIC:                         7.799e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                         3316.5655   7192.099      0.461      0.645   -1.08e+04    1.74e+04
C(sexo)[T.M]                      6385.4406    298.547     21.388      0.000    5800.109    6970.772
posse_de_veiculo[T.True]          -423.1622    282.950     -1.496      0.135    -977.915     131.590
posse_de_imovel[T.True]            225.5227    278.700      0.809      0.418    -320.897     771.943
tipo_renda[T.Empresário]           926.2318    323.760      2.861      0.004     291.467    1560.996
tipo_renda[T.Pensionista]         2690.3196    519.634      5.177      0.000    1671.524    3709.115
tipo_renda[T.Servidor público]    -128.5786    472.495     -0.272      0.786   -1054.953     797.796
educacao[T.Pós graduação]         1680.8059   3406.485      0.493      0.622   -4997.952    8359.564
educacao[T.Secundário]             498.2077   1257.359      0.396      0.692   -1966.972    2963.387
educacao[T.Superior completo]      723.6434   1265.240      0.572      0.567   -1756.987    3204.274
educacao[T.Superior incompleto]   -181.5190   1408.601     -0.129      0.897   -2943.223    2580.185
estado_civil[T.Separado]         -2824.7662   3533.037     -0.800      0.424   -9751.641    4102.109
estado_civil[T.Solteiro]         -3647.4307   3475.884     -1.049      0.294   -1.05e+04    3167.390
estado_civil[T.União]             -448.9957    514.868     -0.872      0.383   -1458.447     560.455
estado_civil[T.Viúvo]            -3542.9743   3548.949     -0.998      0.318   -1.05e+04    3415.098
tipo_residencia[T.Casa]            -94.7471   1149.074     -0.082      0.934   -2347.622    2158.128
tipo_residencia[T.Com os pais]     -15.1621   1277.870     -0.012      0.991   -2520.556    2490.232
tipo_residencia[T.Comunitário]     355.3477   2208.104      0.161      0.872   -3973.864    4684.559
tipo_residencia[T.Estúdio]        1856.0739   2008.462      0.924      0.355   -2081.719    5793.867
tipo_residencia[T.Governamental]   477.2171   1365.617      0.349      0.727   -2200.214    3154.648
qtd_filhos                        2953.1481   3505.619      0.842      0.400   -3919.972    9826.268
idade                               41.8500     16.178      2.587      0.010      10.130      73.569
tempo_emprego                      576.5410     22.251     25.911      0.000     532.916     620.166
qt_pessoas_residencia            -3010.0789   3495.813     -0.861      0.389   -9863.973    3843.815
==============================================================================
Omnibus:                     5313.243   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2370379.061
Skew:                           8.130   Prob(JB):                         0.00
Kurtosis:                     125.090   

In [23]:
reg = model.fit_regularized(method = 'elastic_net'
                           , refit = True
                           , L1_wt = 1
                           , alpha = 0.005)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.253
Model:                            OLS   Adj. R-squared:                  0.249
Method:                 Least Squares   F-statistic:                     52.62
Date:                Mon, 04 Sep 2023   Prob (F-statistic):          4.65e-215
Time:                        09:50:23   Log-Likelihood:                -38891.
No. Observations:                3750   AIC:                         7.783e+04
Df Residuals:                    3726   BIC:                         7.799e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                         3316.5655   7192.099      0.461      0.645   -1.08e+04    1.74e+04
C(sexo)[T.M]                      6385.4406    298.547     21.388      0.000    5800.109    6970.772
posse_de_veiculo[T.True]          -423.1622    282.950     -1.496      0.135    -977.915     131.590
posse_de_imovel[T.True]            225.5227    278.700      0.809      0.418    -320.897     771.943
tipo_renda[T.Empresário]           926.2318    323.760      2.861      0.004     291.467    1560.996
tipo_renda[T.Pensionista]         2690.3196    519.634      5.177      0.000    1671.524    3709.115
tipo_renda[T.Servidor público]    -128.5786    472.495     -0.272      0.786   -1054.953     797.796
educacao[T.Pós graduação]         1680.8059   3406.485      0.493      0.622   -4997.952    8359.564
educacao[T.Secundário]             498.2077   1257.359      0.396      0.692   -1966.972    2963.387
educacao[T.Superior completo]      723.6434   1265.240      0.572      0.567   -1756.987    3204.274
educacao[T.Superior incompleto]   -181.5190   1408.601     -0.129      0.897   -2943.223    2580.185
estado_civil[T.Separado]         -2824.7662   3533.037     -0.800      0.424   -9751.641    4102.109
estado_civil[T.Solteiro]         -3647.4307   3475.884     -1.049      0.294   -1.05e+04    3167.390
estado_civil[T.União]             -448.9957    514.868     -0.872      0.383   -1458.447     560.455
estado_civil[T.Viúvo]            -3542.9743   3548.949     -0.998      0.318   -1.05e+04    3415.098
tipo_residencia[T.Casa]            -94.7471   1149.074     -0.082      0.934   -2347.622    2158.128
tipo_residencia[T.Com os pais]     -15.1621   1277.870     -0.012      0.991   -2520.556    2490.232
tipo_residencia[T.Comunitário]     355.3477   2208.104      0.161      0.872   -3973.864    4684.559
tipo_residencia[T.Estúdio]        1856.0739   2008.462      0.924      0.355   -2081.719    5793.867
tipo_residencia[T.Governamental]   477.2171   1365.617      0.349      0.727   -2200.214    3154.648
qtd_filhos                        2953.1481   3505.619      0.842      0.400   -3919.972    9826.268
idade                               41.8500     16.178      2.587      0.010      10.130      73.569
tempo_emprego                      576.5410     22.251     25.911      0.000     532.916     620.166
qt_pessoas_residencia            -3010.0789   3495.813     -0.861      0.389   -9863.973    3843.815
==============================================================================
Omnibus:                     5313.243   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2370379.061
Skew:                           8.130   Prob(JB):                         0.00
Kurtosis:                     125.090   

In [24]:
reg = model.fit_regularized(method = 'elastic_net'
                           , refit = True
                           , L1_wt = 1
                           , alpha = 0.01)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.253
Model:                            OLS   Adj. R-squared:                  0.249
Method:                 Least Squares   F-statistic:                     52.62
Date:                Mon, 04 Sep 2023   Prob (F-statistic):          4.65e-215
Time:                        09:50:25   Log-Likelihood:                -38891.
No. Observations:                3750   AIC:                         7.783e+04
Df Residuals:                    3726   BIC:                         7.799e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                         3316.5655   7192.099      0.461      0.645   -1.08e+04    1.74e+04
C(sexo)[T.M]                      6385.4406    298.547     21.388      0.000    5800.109    6970.772
posse_de_veiculo[T.True]          -423.1622    282.950     -1.496      0.135    -977.915     131.590
posse_de_imovel[T.True]            225.5227    278.700      0.809      0.418    -320.897     771.943
tipo_renda[T.Empresário]           926.2318    323.760      2.861      0.004     291.467    1560.996
tipo_renda[T.Pensionista]         2690.3196    519.634      5.177      0.000    1671.524    3709.115
tipo_renda[T.Servidor público]    -128.5786    472.495     -0.272      0.786   -1054.953     797.796
educacao[T.Pós graduação]         1680.8059   3406.485      0.493      0.622   -4997.952    8359.564
educacao[T.Secundário]             498.2077   1257.359      0.396      0.692   -1966.972    2963.387
educacao[T.Superior completo]      723.6434   1265.240      0.572      0.567   -1756.987    3204.274
educacao[T.Superior incompleto]   -181.5190   1408.601     -0.129      0.897   -2943.223    2580.185
estado_civil[T.Separado]         -2824.7662   3533.037     -0.800      0.424   -9751.641    4102.109
estado_civil[T.Solteiro]         -3647.4307   3475.884     -1.049      0.294   -1.05e+04    3167.390
estado_civil[T.União]             -448.9957    514.868     -0.872      0.383   -1458.447     560.455
estado_civil[T.Viúvo]            -3542.9743   3548.949     -0.998      0.318   -1.05e+04    3415.098
tipo_residencia[T.Casa]            -94.7471   1149.074     -0.082      0.934   -2347.622    2158.128
tipo_residencia[T.Com os pais]     -15.1621   1277.870     -0.012      0.991   -2520.556    2490.232
tipo_residencia[T.Comunitário]     355.3477   2208.104      0.161      0.872   -3973.864    4684.559
tipo_residencia[T.Estúdio]        1856.0739   2008.462      0.924      0.355   -2081.719    5793.867
tipo_residencia[T.Governamental]   477.2171   1365.617      0.349      0.727   -2200.214    3154.648
qtd_filhos                        2953.1481   3505.619      0.842      0.400   -3919.972    9826.268
idade                               41.8500     16.178      2.587      0.010      10.130      73.569
tempo_emprego                      576.5410     22.251     25.911      0.000     532.916     620.166
qt_pessoas_residencia            -3010.0789   3495.813     -0.861      0.389   -9863.973    3843.815
==============================================================================
Omnibus:                     5313.243   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2370379.061
Skew:                           8.130   Prob(JB):                         0.00
Kurtosis:                     125.090   

In [25]:
reg = model.fit_regularized(method = 'elastic_net'
                           , refit = True
                           , L1_wt = 1
                           , alpha = 0.05)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.253
Model:                            OLS   Adj. R-squared:                  0.249
Method:                 Least Squares   F-statistic:                     54.92
Date:                Mon, 04 Sep 2023   Prob (F-statistic):          6.28e-216
Time:                        09:50:27   Log-Likelihood:                -38891.
No. Observations:                3750   AIC:                         7.783e+04
Df Residuals:                    3727   BIC:                         7.798e+04
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                         3416.4778   7164.316      0.477      0.633   -1.06e+04    1.75e+04
C(sexo)[T.M]                      6383.9980    298.373     21.396      0.000    5799.007    6968.989
posse_de_veiculo[T.True]          -422.4016    282.874     -1.493      0.135    -977.004     132.201
posse_de_imovel[T.True]            226.1019    278.641      0.811      0.417    -320.201     772.405
tipo_renda[T.Empresário]           924.8587    323.606      2.858      0.004     290.397    1559.320
tipo_renda[T.Pensionista]         2686.3388    518.977      5.176      0.000    1668.832    3703.846
tipo_renda[T.Servidor público]    -130.2931    472.313     -0.276      0.783   -1056.311     795.725
educacao[T.Pós graduação]         1667.0198   3404.963      0.490      0.624   -5008.753    8342.793
educacao[T.Secundário]             483.7874   1253.998      0.386      0.700   -1974.802    2942.377
educacao[T.Superior completo]      711.4182   1262.792      0.563      0.573   -1764.413    3187.250
educacao[T.Superior incompleto]   -193.8727   1406.324     -0.138      0.890   -2951.112    2563.367
estado_civil[T.Separado]         -2823.9250   3532.571     -0.799      0.424   -9749.887    4102.037
estado_civil[T.Solteiro]         -3646.7732   3475.427     -1.049      0.294   -1.05e+04    3167.151
estado_civil[T.União]             -449.7486    514.780     -0.874      0.382   -1459.026     559.529
estado_civil[T.Viúvo]            -3537.1982   3548.304     -0.997      0.319   -1.05e+04    3419.609
tipo_residencia[T.Casa]           -189.1101    988.075     -0.191      0.848   -2126.331    1748.111
tipo_residencia[T.Com os pais]    -109.0085   1136.902     -0.096      0.924   -2338.019    2120.002
tipo_residencia[T.Comunitário]            0          0        nan        nan           0           0
tipo_residencia[T.Estúdio]        1762.6238   1922.434      0.917      0.359   -2006.501    5531.749
tipo_residencia[T.Governamental]   382.9844   1233.547      0.310      0.756   -2035.509    2801.477
qtd_filhos                        2951.1665   3505.139      0.842      0.400   -3921.012    9823.345
idade                               41.9672     16.160      2.597      0.009      10.284      73.650
tempo_emprego                      576.4421     22.240     25.920      0.000     532.839     620.045
qt_pessoas_residencia            -3007.9221   3495.331     -0.861      0.390   -9860.870    3845.025
==============================================================================
Omnibus:                     5313.327   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2370573.388
Skew:                           8.131   Prob(JB):                         0.00
Kurtosis:                     125.095   

In [26]:
reg = model.fit_regularized(method = 'elastic_net'
                           , refit = True
                           , L1_wt = 1
                           , alpha = 0.1)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.253
Model:                            OLS   Adj. R-squared:                  0.249
Method:                 Least Squares   F-statistic:                     54.92
Date:                Mon, 04 Sep 2023   Prob (F-statistic):          6.28e-216
Time:                        09:50:28   Log-Likelihood:                -38891.
No. Observations:                3750   AIC:                         7.783e+04
Df Residuals:                    3727   BIC:                         7.798e+04
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                         3416.4778   7164.316      0.477      0.633   -1.06e+04    1.75e+04
C(sexo)[T.M]                      6383.9980    298.373     21.396      0.000    5799.007    6968.989
posse_de_veiculo[T.True]          -422.4016    282.874     -1.493      0.135    -977.004     132.201
posse_de_imovel[T.True]            226.1019    278.641      0.811      0.417    -320.201     772.405
tipo_renda[T.Empresário]           924.8587    323.606      2.858      0.004     290.397    1559.320
tipo_renda[T.Pensionista]         2686.3388    518.977      5.176      0.000    1668.832    3703.846
tipo_renda[T.Servidor público]    -130.2931    472.313     -0.276      0.783   -1056.311     795.725
educacao[T.Pós graduação]         1667.0198   3404.963      0.490      0.624   -5008.753    8342.793
educacao[T.Secundário]             483.7874   1253.998      0.386      0.700   -1974.802    2942.377
educacao[T.Superior completo]      711.4182   1262.792      0.563      0.573   -1764.413    3187.250
educacao[T.Superior incompleto]   -193.8727   1406.324     -0.138      0.890   -2951.112    2563.367
estado_civil[T.Separado]         -2823.9250   3532.571     -0.799      0.424   -9749.887    4102.037
estado_civil[T.Solteiro]         -3646.7732   3475.427     -1.049      0.294   -1.05e+04    3167.151
estado_civil[T.União]             -449.7486    514.780     -0.874      0.382   -1459.026     559.529
estado_civil[T.Viúvo]            -3537.1982   3548.304     -0.997      0.319   -1.05e+04    3419.609
tipo_residencia[T.Casa]           -189.1101    988.075     -0.191      0.848   -2126.331    1748.111
tipo_residencia[T.Com os pais]    -109.0085   1136.902     -0.096      0.924   -2338.019    2120.002
tipo_residencia[T.Comunitário]            0          0        nan        nan           0           0
tipo_residencia[T.Estúdio]        1762.6238   1922.434      0.917      0.359   -2006.501    5531.749
tipo_residencia[T.Governamental]   382.9844   1233.547      0.310      0.756   -2035.509    2801.477
qtd_filhos                        2951.1665   3505.139      0.842      0.400   -3921.012    9823.345
idade                               41.9672     16.160      2.597      0.009      10.284      73.650
tempo_emprego                      576.4421     22.240     25.920      0.000     532.839     620.045
qt_pessoas_residencia            -3007.9221   3495.331     -0.861      0.390   -9860.870    3845.025
==============================================================================
Omnibus:                     5313.327   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2370573.388
Skew:                           8.131   Prob(JB):                         0.00
Kurtosis:                     125.095   

- Os valores de $R^2$ não apresentaram alterações.

4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?

In [91]:
#Gerando Dummies
df_test_encoded = pd.get_dummies(df_test.drop('renda', axis=1), columns = ['sexo', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia'])

In [92]:
#Checando se os tipos de dados são todos compatíveis com o modelo stepwise.
df_test_encoded.dtypes

posse_de_veiculo                    bool
posse_de_imovel                     bool
qtd_filhos                         int64
idade                              int64
tempo_emprego                    float64
qt_pessoas_residencia            float64
sexo_F                             uint8
sexo_M                             uint8
tipo_renda_Assalariado             uint8
tipo_renda_Empresário              uint8
tipo_renda_Pensionista             uint8
tipo_renda_Servidor público        uint8
educacao_Primário                  uint8
educacao_Pós graduação             uint8
educacao_Secundário                uint8
educacao_Superior completo         uint8
educacao_Superior incompleto       uint8
estado_civil_Casado                uint8
estado_civil_Separado              uint8
estado_civil_Solteiro              uint8
estado_civil_União                 uint8
estado_civil_Viúvo                 uint8
tipo_residencia_Aluguel            uint8
tipo_residencia_Casa               uint8
tipo_residencia_

- OBS.: após consecutivos testes, percebi que o código do stepwise não é compatível com valores booleanos. Por isso, decidi converter essas colunas para valores inteiros a fim não perder essas informações.

In [96]:
#Convertendo colunas:
df_test_encoded.posse_de_veiculo = [int(x) for x in df_test_encoded.posse_de_veiculo]
df_test_encoded.posse_de_imovel = [int(x) for x in df_test_encoded.posse_de_imovel]

In [97]:
#Separando variáveis:
X = df_test_encoded
y = df_test.renda

In [98]:
#Algoritimo do stepwise
def stepwise_selection(X, y,

                       initial_list=[],

                       threshold_in=0.01,

                       threshold_out = 0.05,

                       verbose=True):

    """ Perform a forward-backward feature selection

    based on p-value from statsmodels.api.OLS

    Arguments:

        X - pandas.DataFrame with candidate features

        y - list-like with the target

        initial_list - list of features to start with (column names of X)

        threshold_in - include a feature if its p-value < threshold_in

        threshold_out - exclude a feature if its p-value > threshold_out

        verbose - whether to print the sequence of inclusions and exclusions

    Returns: list of selected features

    Always set threshold_in < threshold_out to avoid infinite looping.

    See https://en.wikipedia.org/wiki/Stepwise_regression for the details

    """

    included = list(initial_list)

    while True:

        changed=False

        # forward step

        excluded = list(set(X.columns)-set(included))

        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))

        for new_column in excluded:

            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()

            new_pval[new_column] = model.pvalues[new_column]

        best_pval = new_pval.min()

        if best_pval < threshold_in:

            best_feature = new_pval.index[new_pval.argmin()]

            included.append(best_feature)

            changed=True

            if verbose:

                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))



        # backward step

        print("#############")

        print(included)

        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()

        # use all coefs except intercept

        pvalues = model.pvalues.iloc[1:]

        worst_pval = pvalues.max() # null if pvalues is empty

        if worst_pval > threshold_out:

            changed=True

            worst_feature = pvalues.argmax()

            included.remove(worst_feature)

            if verbose:

                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))

        if not changed:

            break

    return included

In [99]:
#Chamando função do stepwise
variaveis = stepwise_selection(X, y)

#Imprimindo resultados:
print('resulting features:')
print(variaveis)

Add  tempo_emprego                  with p-value 1.82385e-129
#############
['tempo_emprego']
Add  sexo_F                         with p-value 5.31222e-89
#############
['tempo_emprego', 'sexo_F']
Add  sexo_M                         with p-value 1.1031e-124
#############
['tempo_emprego', 'sexo_F', 'sexo_M']
Add  tipo_renda_Pensionista         with p-value 1.81333e-18
#############
['tempo_emprego', 'sexo_F', 'sexo_M', 'tipo_renda_Pensionista']
Add  idade                          with p-value 0.00148964
#############
['tempo_emprego', 'sexo_F', 'sexo_M', 'tipo_renda_Pensionista', 'idade']
Add  tipo_renda_Empresário          with p-value 0.00253948
#############
['tempo_emprego', 'sexo_F', 'sexo_M', 'tipo_renda_Pensionista', 'idade', 'tipo_renda_Empresário']
#############
['tempo_emprego', 'sexo_F', 'sexo_M', 'tipo_renda_Pensionista', 'idade', 'tipo_renda_Empresário']
resulting features:
['tempo_emprego', 'sexo_F', 'sexo_M', 'tipo_renda_Pensionista', 'idade', 'tipo_renda_Empresário']


In [103]:
#Adicionando a coluna 'renda' para poder executar a regressão:
df_test_encoded = df_test_encoded.assign(renda = y)

In [105]:
modelo = smf.ols('renda ~ tempo_emprego + sexo_F + sexo_M + tipo_renda_Pensionista + idade + tipo_renda_Empresário', data = df_test_encoded)
reg = model.fit_regularized(method = 'elastic_net'
                        , refit = True
                        , L1_wt = 0.01
                        , alpha = 0)
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.253
Model:                            OLS   Adj. R-squared:                  0.249
Method:                 Least Squares   F-statistic:                     52.62
Date:                Mon, 04 Sep 2023   Prob (F-statistic):          4.65e-215
Time:                        16:02:23   Log-Likelihood:                -38891.
No. Observations:                3750   AIC:                         7.783e+04
Df Residuals:                    3726   BIC:                         7.799e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                         3316.5655   7192.099      0.461      0.645   -1.08e+04    1.74e+04
C(sexo)[T.M]                      6385.4406    298.547     21.388      0.000    5800.109    6970.772
posse_de_veiculo[T.True]          -423.1622    282.950     -1.496      0.135    -977.915     131.590
posse_de_imovel[T.True]            225.5227    278.700      0.809      0.418    -320.897     771.943
tipo_renda[T.Empresário]           926.2318    323.760      2.861      0.004     291.467    1560.996
tipo_renda[T.Pensionista]         2690.3196    519.634      5.177      0.000    1671.524    3709.115
tipo_renda[T.Servidor público]    -128.5786    472.495     -0.272      0.786   -1054.953     797.796
educacao[T.Pós graduação]         1680.8059   3406.485      0.493      0.622   -4997.952    8359.564
educacao[T.Secundário]             498.2077   1257.359      0.396      0.692   -1966.972    2963.387
educacao[T.Superior completo]      723.6434   1265.240      0.572      0.567   -1756.987    3204.274
educacao[T.Superior incompleto]   -181.5190   1408.601     -0.129      0.897   -2943.223    2580.185
estado_civil[T.Separado]         -2824.7662   3533.037     -0.800      0.424   -9751.641    4102.109
estado_civil[T.Solteiro]         -3647.4307   3475.884     -1.049      0.294   -1.05e+04    3167.390
estado_civil[T.União]             -448.9957    514.868     -0.872      0.383   -1458.447     560.455
estado_civil[T.Viúvo]            -3542.9743   3548.949     -0.998      0.318   -1.05e+04    3415.098
tipo_residencia[T.Casa]            -94.7471   1149.074     -0.082      0.934   -2347.622    2158.128
tipo_residencia[T.Com os pais]     -15.1621   1277.870     -0.012      0.991   -2520.556    2490.232
tipo_residencia[T.Comunitário]     355.3477   2208.104      0.161      0.872   -3973.864    4684.559
tipo_residencia[T.Estúdio]        1856.0739   2008.462      0.924      0.355   -2081.719    5793.867
tipo_residencia[T.Governamental]   477.2171   1365.617      0.349      0.727   -2200.214    3154.648
qtd_filhos                        2953.1481   3505.619      0.842      0.400   -3919.972    9826.268
idade                               41.8500     16.178      2.587      0.010      10.130      73.569
tempo_emprego                      576.5410     22.251     25.911      0.000     532.916     620.166
qt_pessoas_residencia            -3010.0789   3495.813     -0.861      0.389   -9863.973    3843.815
==============================================================================
Omnibus:                     5313.243   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2370379.061
Skew:                           8.130   Prob(JB):                         0.00
Kurtosis:                     125.090   

- Não houve alterações nos valores de $R^2$.

5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?

- Não foi possível notar diferenças entres os modelos gerados até o momento.  

6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.

In [108]:
modelo = smf.ols('np.log(renda) ~ tempo_emprego + sexo_F + sexo_M + tipo_renda_Pensionista + np.log(idade) + tipo_renda_Empresário', data = df_test_encoded)
reg = model.fit_regularized(method = 'elastic_net'
                        , refit = True
                        , L1_wt = 0.01
                        , alpha = 0)
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.253
Model:                            OLS   Adj. R-squared:                  0.249
Method:                 Least Squares   F-statistic:                     52.62
Date:                Mon, 04 Sep 2023   Prob (F-statistic):          4.65e-215
Time:                        16:06:22   Log-Likelihood:                -38891.
No. Observations:                3750   AIC:                         7.783e+04
Df Residuals:                    3726   BIC:                         7.799e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                         3316.5655   7192.099      0.461      0.645   -1.08e+04    1.74e+04
C(sexo)[T.M]                      6385.4406    298.547     21.388      0.000    5800.109    6970.772
posse_de_veiculo[T.True]          -423.1622    282.950     -1.496      0.135    -977.915     131.590
posse_de_imovel[T.True]            225.5227    278.700      0.809      0.418    -320.897     771.943
tipo_renda[T.Empresário]           926.2318    323.760      2.861      0.004     291.467    1560.996
tipo_renda[T.Pensionista]         2690.3196    519.634      5.177      0.000    1671.524    3709.115
tipo_renda[T.Servidor público]    -128.5786    472.495     -0.272      0.786   -1054.953     797.796
educacao[T.Pós graduação]         1680.8059   3406.485      0.493      0.622   -4997.952    8359.564
educacao[T.Secundário]             498.2077   1257.359      0.396      0.692   -1966.972    2963.387
educacao[T.Superior completo]      723.6434   1265.240      0.572      0.567   -1756.987    3204.274
educacao[T.Superior incompleto]   -181.5190   1408.601     -0.129      0.897   -2943.223    2580.185
estado_civil[T.Separado]         -2824.7662   3533.037     -0.800      0.424   -9751.641    4102.109
estado_civil[T.Solteiro]         -3647.4307   3475.884     -1.049      0.294   -1.05e+04    3167.390
estado_civil[T.União]             -448.9957    514.868     -0.872      0.383   -1458.447     560.455
estado_civil[T.Viúvo]            -3542.9743   3548.949     -0.998      0.318   -1.05e+04    3415.098
tipo_residencia[T.Casa]            -94.7471   1149.074     -0.082      0.934   -2347.622    2158.128
tipo_residencia[T.Com os pais]     -15.1621   1277.870     -0.012      0.991   -2520.556    2490.232
tipo_residencia[T.Comunitário]     355.3477   2208.104      0.161      0.872   -3973.864    4684.559
tipo_residencia[T.Estúdio]        1856.0739   2008.462      0.924      0.355   -2081.719    5793.867
tipo_residencia[T.Governamental]   477.2171   1365.617      0.349      0.727   -2200.214    3154.648
qtd_filhos                        2953.1481   3505.619      0.842      0.400   -3919.972    9826.268
idade                               41.8500     16.178      2.587      0.010      10.130      73.569
tempo_emprego                      576.5410     22.251     25.911      0.000     532.916     620.166
qt_pessoas_residencia            -3010.0789   3495.813     -0.861      0.389   -9863.973    3843.815
==============================================================================
Omnibus:                     5313.243   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2370379.061
Skew:                           8.130   Prob(JB):                         0.00
Kurtosis:                     125.090   

7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [222]:
#Definindo árvore
arvore_1 = DecisionTreeRegressor(max_depth=18) #Testei vários valores de profundidade até chegar neste valor ideal.

In [223]:
#Treinando árvore
arvore_1.fit(X, y)

DecisionTreeRegressor(max_depth=18)

In [224]:
print(f"R-quadrado na base de testes: {arvore_1.score(X, y):.2f}")

R-quadrado na base de testes: 0.85


- Através da Árvore de Regressão foi possível alcançar valores de $R^2$ muito superiores à qualquer outro modelo de regressão feito anteriormente.